<a href="https://colab.research.google.com/github/k2-fsa/colab/blob/master/sherpa-onnx/sherpa_onnx_python_non_streaming_websocket_server.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction

This colab notebook shows how to run the Python non-streaming WebSocket server in [sherpa-onnx](https://github.com/k2-fsa/sherpa-onnx)

Please refer to https://k2-fsa.github.io/sherpa/onnx/python/non-streaming-websocket-server.html for details.





# Install sherpa-onnx


In [1]:
%%shell

pip install sherpa-onnx
pip install websockets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 85.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 51.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 3.3 MB/s eta 0:00:00


We use the following types of models for demonstration.

|Description|URL|
|---|---|
|Non-streaming transducer|[csukuangfj/sherpa-onnx-zipformer-en-2023-06-26 (English)](http://localhost:6007/onnx/pretrained_models/offline-transducer/zipformer-transducer-models.html#sherpa-onnx-zipformer-en-2023-06-26-english)|
|Non-streaming paraformer|[csukuangfj/sherpa-onnx-paraformer-zh-2023-03-28 (Chinese)](http://localhost:6007/onnx/pretrained_models/offline-paraformer/paraformer-models.html#sherpa-onnx-offline-paraformer-zh-2023-03-28-chinese)|
|Non-streaming CTC model from NeMo|[stt_en_conformer_ctc_medium](http://localhost:6007/onnx/pretrained_models/offline-ctc/nemo/english.html#stt-en-conformer-ctc-medium-nemo-sherpa-onnx)|
|Non-streaming Whisper tiny.en|[tiny.en](http://localhost:6007/onnx/pretrained_models/whisper/tiny.en.html#whisper-tiny-en-sherpa-onnx)|

We download the python example code first:

In [2]:
%%shell

git clone https://github.com/k2-fsa/sherpa-onnx

Cloning into 'sherpa-onnx'...
remote: Enumerating objects: 2672, done.
remote: Counting objects: 100% (1149/1149), done.
remote: Compressing objects: 100% (456/456), done.
remote: Total 2672 (delta 807), reused 848 (delta 682), pack-reused 1523
Receiving objects: 100% (2672/2672), 1.92 MiB | 23.40 MiB/s, done.
Resolving deltas: 100% (1539/1539), done.


## Non-streaming transducer

### Start the server

In [3]:
%%shell

cd sherpa-onnx

GIT_LFS_SKIP_SMUDGE=1 git clone https://huggingface.co/csukuangfj/sherpa-onnx-zipformer-en-2023-06-26
cd sherpa-onnx-zipformer-en-2023-06-26
git lfs pull --include "*.onnx"
cd ..

nohup python3 ./python-api-examples/non_streaming_server.py \
  --encoder ./sherpa-onnx-zipformer-en-2023-06-26/encoder-epoch-99-avg-1.onnx \
  --decoder ./sherpa-onnx-zipformer-en-2023-06-26/decoder-epoch-99-avg-1.onnx \
  --joiner ./sherpa-onnx-zipformer-en-2023-06-26/joiner-epoch-99-avg-1.onnx \
  --tokens ./sherpa-onnx-zipformer-en-2023-06-26/tokens.txt \
  --port 6006 > non-streaming-transducer.log &

Cloning into 'sherpa-onnx-zipformer-en-2023-06-26'...
remote: Enumerating objects: 20, done.
remote: Total 20 (delta 0), reused 0 (delta 0), pack-reused 20
Unpacking objects: 100% (20/20), 666.78 KiB | 11.11 MiB/s, done.
nohup: redirecting stderr to stdout


### Start the client

#### Decode multiple files in parallel



In [4]:
%%shell

# sleep 10 second to wait for the server startup
sleep 10

cd sherpa-onnx

python3 ./python-api-examples/offline-websocket-client-decode-files-paralell.py \
  --server-addr localhost \
  --server-port 6006 \
  ./sherpa-onnx-zipformer-en-2023-06-26/test_wavs/0.wav \
  ./sherpa-onnx-zipformer-en-2023-06-26/test_wavs/1.wav \
  ./sherpa-onnx-zipformer-en-2023-06-26/test_wavs/8k.wav

2023-08-11 10:56:41,931 INFO [offline-websocket-client-decode-files-paralell.py:139] {'server_addr': 'localhost', 'server_port': 6006, 'sound_files': ['./sherpa-onnx-zipformer-en-2023-06-26/test_wavs/0.wav', './sherpa-onnx-zipformer-en-2023-06-26/test_wavs/1.wav', './sherpa-onnx-zipformer-en-2023-06-26/test_wavs/8k.wav']}
2023-08-11 10:56:41,953 INFO [offline-websocket-client-decode-files-paralell.py:113] Sending ./sherpa-onnx-zipformer-en-2023-06-26/test_wavs/0.wav
2023-08-11 10:56:42,000 INFO [offline-websocket-client-decode-files-paralell.py:113] Sending ./sherpa-onnx-zipformer-en-2023-06-26/test_wavs/1.wav
2023-08-11 10:56:42,174 INFO [offline-websocket-client-decode-files-paralell.py:113] Sending ./sherpa-onnx-zipformer-en-2023-06-26/test_wavs/8k.wav
2023-08-11 10:56:42,629 INFO [offline-websocket-client-decode-files-paralell.py:131] ./sherpa-onnx-zipformer-en-2023-06-26/test_wavs/0.wav
 AFTER EARLY NIGHTFALL THE YELLOW LAMPS WOULD LIGHT UP HERE AND THERE THE SQUALID QUARTER OF TH

#### Decode multiple files sequentially



In [5]:
%%shell

cd sherpa-onnx

python3 ./python-api-examples/offline-websocket-client-decode-files-sequential.py \
  --server-addr localhost \
  --server-port 6006 \
  ./sherpa-onnx-zipformer-en-2023-06-26/test_wavs/0.wav \
  ./sherpa-onnx-zipformer-en-2023-06-26/test_wavs/1.wav \
  ./sherpa-onnx-zipformer-en-2023-06-26/test_wavs/8k.wav

2023-08-11 10:56:43,576 INFO [offline-websocket-client-decode-files-sequential.py:141] {'server_addr': 'localhost', 'server_port': 6006, 'sound_files': ['./sherpa-onnx-zipformer-en-2023-06-26/test_wavs/0.wav', './sherpa-onnx-zipformer-en-2023-06-26/test_wavs/1.wav', './sherpa-onnx-zipformer-en-2023-06-26/test_wavs/8k.wav']}
2023-08-11 10:56:43,595 INFO [offline-websocket-client-decode-files-sequential.py:114] Sending ./sherpa-onnx-zipformer-en-2023-06-26/test_wavs/0.wav
 AFTER EARLY NIGHTFALL THE YELLOW LAMPS WOULD LIGHT UP HERE AND THERE THE SQUALID QUARTER OF THE BROTHELS
2023-08-11 10:56:43,912 INFO [offline-websocket-client-decode-files-sequential.py:114] Sending ./sherpa-onnx-zipformer-en-2023-06-26/test_wavs/1.wav
 GOD AS A DIRECT CONSEQUENCE OF THE SIN WHICH MAN THUS PUNISHED HAD GIVEN HER A LOVELY CHILD WHOSE PLACE WAS ON THAT SAME DISHONORED BOSOM TO CONNECT HER PARENT FOREVER WITH THE RACE AND DESCENT OF MORTALS AND TO BE FINALLY A BLESSED SOUL IN HEAVEN
2023-08-11 10:56:44,7

## Non-streaming paraformer

### Start the server

In [6]:
%%shell

cd sherpa-onnx

GIT_LFS_SKIP_SMUDGE=1 git clone https://huggingface.co/csukuangfj/sherpa-onnx-paraformer-zh-2023-03-28
cd sherpa-onnx-paraformer-zh-2023-03-28
git lfs pull --include "*.onnx"
cd ..

nohup python3 ./python-api-examples/non_streaming_server.py \
  --paraformer ./sherpa-onnx-paraformer-zh-2023-03-28/model.int8.onnx \
  --tokens ./sherpa-onnx-paraformer-zh-2023-03-28/tokens.txt \
  --port 6007 > non-streaming-paraformer.log &

Cloning into 'sherpa-onnx-paraformer-zh-2023-03-28'...
remote: Enumerating objects: 17, done.
remote: Total 17 (delta 0), reused 0 (delta 0), pack-reused 17
Unpacking objects: 100% (17/17), 491.76 KiB | 15.86 MiB/s, done.
nohup: redirecting stderr to stdout


### Start the client

#### Decode multiple files in parallel



In [7]:
%%shell

sleep 10
cd sherpa-onnx

python3 ./python-api-examples/offline-websocket-client-decode-files-paralell.py \
  --server-addr localhost \
  --server-port 6007 \
  ./sherpa-onnx-paraformer-zh-2023-03-28/test_wavs/0.wav \
  ./sherpa-onnx-paraformer-zh-2023-03-28/test_wavs/1.wav \
  ./sherpa-onnx-paraformer-zh-2023-03-28/test_wavs/2.wav \
  ./sherpa-onnx-paraformer-zh-2023-03-28/test_wavs/8k.wav

2023-08-11 10:57:16,459 INFO [offline-websocket-client-decode-files-paralell.py:139] {'server_addr': 'localhost', 'server_port': 6007, 'sound_files': ['./sherpa-onnx-paraformer-zh-2023-03-28/test_wavs/0.wav', './sherpa-onnx-paraformer-zh-2023-03-28/test_wavs/1.wav', './sherpa-onnx-paraformer-zh-2023-03-28/test_wavs/2.wav', './sherpa-onnx-paraformer-zh-2023-03-28/test_wavs/8k.wav']}
2023-08-11 10:57:16,482 INFO [offline-websocket-client-decode-files-paralell.py:113] Sending ./sherpa-onnx-paraformer-zh-2023-03-28/test_wavs/0.wav
2023-08-11 10:57:16,522 INFO [offline-websocket-client-decode-files-paralell.py:113] Sending ./sherpa-onnx-paraformer-zh-2023-03-28/test_wavs/1.wav
2023-08-11 10:57:16,569 INFO [offline-websocket-client-decode-files-paralell.py:113] Sending ./sherpa-onnx-paraformer-zh-2023-03-28/test_wavs/2.wav
2023-08-11 10:57:16,632 INFO [offline-websocket-client-decode-files-paralell.py:113] Sending ./sherpa-onnx-paraformer-zh-2023-03-28/test_wavs/8k.wav
2023-08-11 10:57:17,80

#### Decode multiple files sequentially



In [8]:
%%shell

cd sherpa-onnx

python3 ./python-api-examples/offline-websocket-client-decode-files-sequential.py \
  --server-addr localhost \
  --server-port 6007 \
  ./sherpa-onnx-paraformer-zh-2023-03-28/test_wavs/0.wav \
  ./sherpa-onnx-paraformer-zh-2023-03-28/test_wavs/1.wav \
  ./sherpa-onnx-paraformer-zh-2023-03-28/test_wavs/2.wav \
  ./sherpa-onnx-paraformer-zh-2023-03-28/test_wavs/8k.wav

2023-08-11 10:57:17,996 INFO [offline-websocket-client-decode-files-sequential.py:141] {'server_addr': 'localhost', 'server_port': 6007, 'sound_files': ['./sherpa-onnx-paraformer-zh-2023-03-28/test_wavs/0.wav', './sherpa-onnx-paraformer-zh-2023-03-28/test_wavs/1.wav', './sherpa-onnx-paraformer-zh-2023-03-28/test_wavs/2.wav', './sherpa-onnx-paraformer-zh-2023-03-28/test_wavs/8k.wav']}
2023-08-11 10:57:18,015 INFO [offline-websocket-client-decode-files-sequential.py:114] Sending ./sherpa-onnx-paraformer-zh-2023-03-28/test_wavs/0.wav
对我做了介绍啊那么我想说的是呢大家如果对我的研究感兴趣呢你
2023-08-11 10:57:18,406 INFO [offline-websocket-client-decode-files-sequential.py:114] Sending ./sherpa-onnx-paraformer-zh-2023-03-28/test_wavs/1.wav
重点呢想谈三个问题首先呢就是这一轮全球金融动荡的表现
2023-08-11 10:57:18,746 INFO [offline-websocket-client-decode-files-sequential.py:114] Sending ./sherpa-onnx-paraformer-zh-2023-03-28/test_wavs/2.wav
深入的分析这一次全球金融动荡背后的根源
2023-08-11 10:57:19,055 INFO [offline-websocket-client-decode-files-sequential.py:114]

## Non-streaming CTC model from NeMo

### Start the server

In [9]:
%%shell

cd sherpa-onnx

GIT_LFS_SKIP_SMUDGE=1 git clone https://huggingface.co/csukuangfj/sherpa-onnx-nemo-ctc-en-conformer-medium
cd sherpa-onnx-nemo-ctc-en-conformer-medium
git lfs pull --include "*.onnx"
cd ..

nohup python3 ./python-api-examples/non_streaming_server.py \
  --nemo-ctc ./sherpa-onnx-nemo-ctc-en-conformer-medium/model.onnx \
  --tokens ./sherpa-onnx-nemo-ctc-en-conformer-medium/tokens.txt \
  --port 6008 > nemo-ctc.log &

Cloning into 'sherpa-onnx-nemo-ctc-en-conformer-medium'...
remote: Enumerating objects: 20, done.
remote: Total 20 (delta 0), reused 0 (delta 0), pack-reused 20
Unpacking objects: 100% (20/20), 671.36 KiB | 15.98 MiB/s, done.
nohup: redirecting stderr to stdout


### Start the client

#### Decode multiple files in parallel

In [10]:
%%shell

sleep 10
cd sherpa-onnx

python3 ./python-api-examples/offline-websocket-client-decode-files-paralell.py \
  --server-addr localhost \
  --server-port 6008 \
  ./sherpa-onnx-nemo-ctc-en-conformer-medium/test_wavs/0.wav \
  ./sherpa-onnx-nemo-ctc-en-conformer-medium/test_wavs/1.wav \
  ./sherpa-onnx-nemo-ctc-en-conformer-medium/test_wavs/8k.wav

2023-08-11 10:57:33,709 INFO [offline-websocket-client-decode-files-paralell.py:139] {'server_addr': 'localhost', 'server_port': 6008, 'sound_files': ['./sherpa-onnx-nemo-ctc-en-conformer-medium/test_wavs/0.wav', './sherpa-onnx-nemo-ctc-en-conformer-medium/test_wavs/1.wav', './sherpa-onnx-nemo-ctc-en-conformer-medium/test_wavs/8k.wav']}
2023-08-11 10:57:33,733 INFO [offline-websocket-client-decode-files-paralell.py:113] Sending ./sherpa-onnx-nemo-ctc-en-conformer-medium/test_wavs/0.wav
2023-08-11 10:57:33,783 INFO [offline-websocket-client-decode-files-paralell.py:113] Sending ./sherpa-onnx-nemo-ctc-en-conformer-medium/test_wavs/1.wav
2023-08-11 10:57:34,026 INFO [offline-websocket-client-decode-files-paralell.py:113] Sending ./sherpa-onnx-nemo-ctc-en-conformer-medium/test_wavs/8k.wav
2023-08-11 10:57:34,304 INFO [offline-websocket-client-decode-files-paralell.py:131] ./sherpa-onnx-nemo-ctc-en-conformer-medium/test_wavs/0.wav
 after early nightfall the yellow lamps would light up here 

#### Decode multiple files sequentially



In [11]:
%%shell

cd sherpa-onnx

python3 ./python-api-examples/offline-websocket-client-decode-files-sequential.py \
  --server-addr localhost \
  --server-port 6008 \
  ./sherpa-onnx-nemo-ctc-en-conformer-medium/test_wavs/0.wav \
  ./sherpa-onnx-nemo-ctc-en-conformer-medium/test_wavs/1.wav \
  ./sherpa-onnx-nemo-ctc-en-conformer-medium/test_wavs/8k.wav

2023-08-11 10:57:35,197 INFO [offline-websocket-client-decode-files-sequential.py:141] {'server_addr': 'localhost', 'server_port': 6008, 'sound_files': ['./sherpa-onnx-nemo-ctc-en-conformer-medium/test_wavs/0.wav', './sherpa-onnx-nemo-ctc-en-conformer-medium/test_wavs/1.wav', './sherpa-onnx-nemo-ctc-en-conformer-medium/test_wavs/8k.wav']}
2023-08-11 10:57:35,216 INFO [offline-websocket-client-decode-files-sequential.py:114] Sending ./sherpa-onnx-nemo-ctc-en-conformer-medium/test_wavs/0.wav
 after early nightfall the yellow lamps would light up here and there the squalid quarter of the brothels
2023-08-11 10:57:35,518 INFO [offline-websocket-client-decode-files-sequential.py:114] Sending ./sherpa-onnx-nemo-ctc-en-conformer-medium/test_wavs/1.wav
 god as a direct consequence of the sin which man thus punished had given her a lovely child whose place was on that same dishonored bosom to connect her parent for ever with the race and descent of mortals and to be finally a blessed soul in he

## Non-streaming Whisper tiny.en

### Start the server

In [12]:
%%shell

cd sherpa-onnx
GIT_LFS_SKIP_SMUDGE=1 git clone https://huggingface.co/csukuangfj/sherpa-onnx-whisper-tiny.en
cd sherpa-onnx-whisper-tiny.en
git lfs pull --include "*.onnx"
cd ..

nohup python3 ./python-api-examples/non_streaming_server.py \
  --whisper-encoder=./sherpa-onnx-whisper-tiny.en/tiny.en-encoder.onnx \
  --whisper-decoder=./sherpa-onnx-whisper-tiny.en/tiny.en-decoder.onnx \
  --tokens=./sherpa-onnx-whisper-tiny.en/tiny.en-tokens.txt \
  --port 6009  > whisper-tiny.en.log &

Cloning into 'sherpa-onnx-whisper-tiny.en'...
remote: Enumerating objects: 30, done.
remote: Counting objects: 100% (30/30), done.
remote: Compressing objects: 100% (29/29), done.
remote: Total 30 (delta 3), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (30/30), 1023.85 KiB | 9.06 MiB/s, done.
nohup: redirecting stderr to stdout


### Start the client

#### Decode multiple files in parallel



In [13]:
%%shell

sleep 10
cd sherpa-onnx

python3 ./python-api-examples/offline-websocket-client-decode-files-paralell.py \
  --server-addr localhost \
  --server-port 6009 \
  ./sherpa-onnx-whisper-tiny.en/test_wavs/0.wav \
  ./sherpa-onnx-whisper-tiny.en/test_wavs/1.wav \
  ./sherpa-onnx-whisper-tiny.en/test_wavs/8k.wav

2023-08-11 10:57:54,399 INFO [offline-websocket-client-decode-files-paralell.py:139] {'server_addr': 'localhost', 'server_port': 6009, 'sound_files': ['./sherpa-onnx-whisper-tiny.en/test_wavs/0.wav', './sherpa-onnx-whisper-tiny.en/test_wavs/1.wav', './sherpa-onnx-whisper-tiny.en/test_wavs/8k.wav']}
2023-08-11 10:57:54,423 INFO [offline-websocket-client-decode-files-paralell.py:113] Sending ./sherpa-onnx-whisper-tiny.en/test_wavs/0.wav
2023-08-11 10:57:54,475 INFO [offline-websocket-client-decode-files-paralell.py:113] Sending ./sherpa-onnx-whisper-tiny.en/test_wavs/1.wav
2023-08-11 10:57:55,633 INFO [offline-websocket-client-decode-files-paralell.py:113] Sending ./sherpa-onnx-whisper-tiny.en/test_wavs/8k.wav
2023-08-11 10:57:55,659 INFO [offline-websocket-client-decode-files-paralell.py:131] ./sherpa-onnx-whisper-tiny.en/test_wavs/0.wav
 After early nightfall, the yellow lamps would light up here and there, the squalid quarter of the brothels.
2023-08-11 10:57:57,619 INFO [offline-webs

#### Decode multiple files sequentially



In [14]:
%%shell

cd sherpa-onnx

python3 ./python-api-examples/offline-websocket-client-decode-files-sequential.py \
  --server-addr localhost \
  --server-port 6009 \
  ./sherpa-onnx-whisper-tiny.en/test_wavs/0.wav \
  ./sherpa-onnx-whisper-tiny.en/test_wavs/1.wav \
  ./sherpa-onnx-whisper-tiny.en/test_wavs/8k.wav

2023-08-11 10:57:58,701 INFO [offline-websocket-client-decode-files-sequential.py:141] {'server_addr': 'localhost', 'server_port': 6009, 'sound_files': ['./sherpa-onnx-whisper-tiny.en/test_wavs/0.wav', './sherpa-onnx-whisper-tiny.en/test_wavs/1.wav', './sherpa-onnx-whisper-tiny.en/test_wavs/8k.wav']}
2023-08-11 10:57:58,724 INFO [offline-websocket-client-decode-files-sequential.py:114] Sending ./sherpa-onnx-whisper-tiny.en/test_wavs/0.wav
 After early nightfall, the yellow lamps would light up here and there, the squalid quarter of the brothels.
2023-08-11 10:57:59,992 INFO [offline-websocket-client-decode-files-sequential.py:114] Sending ./sherpa-onnx-whisper-tiny.en/test_wavs/1.wav
 God, as a direct consequence of the sin which man thus punished, had given her a lovely child, whose place was on that same dishonored bosom to connect her parent forever with the race and descent of mortals, and to be finally a blessed soul in heaven.
2023-08-11 10:58:01,930 INFO [offline-websocket-clien